<font color='lightblue'>Nom Etudiant 1: ELLIETHY Ossama 28712429
<br>
Nom Etudiant 2: HORCHANI Samy 28706765 </font>

# TME 3.1 : Greedy Profile Motif Search


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons d'abord implémenter l'algorithm Greedy Profile Motif Search.


1\. Nous allons réutiliser les fonctions du precedent pour générer `t` séquences artificielles de taille `n`, et implanter dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement. Nous allons faire varier le motifs dans 50% de cas.

In [2]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=80 #longuer des sequence
f= 0.5 #frequence de variation du motif.


In [3]:
from math import floor

def insertMotif(sequence, motif, position): #repris du tme 2
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront en majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    
    for i in range(n):
        s = random.choice(nuc)

        if(upper):
            sequence = sequence + s.upper()
        else:
            sequence = sequence + s.lower()

    return sequence

def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)

    pos_modifie = []
    for i in range(nbpos):
        random_pos = random.randint(0,len(motifM)-1)

        while(random_pos in pos_modifie):
            random_pos = random.randint(0,len(motifM)-1)
        
        pos_modifie.append(random_pos) #sauvegarde de la position pour pas la modifier 2 fois
        random_nuc = random.choice(nuc)

        while(random_nuc == motif[random_pos]): #pour pas implanter le meme motif 
            random_nuc = random.choice(nuc)

        if(upper):
            motifM[random_pos] = random_nuc.upper()
        else:
            motifM[random_pos] = random_nuc.lower()
            
    return "".join(motifM)

#tester la fonction modifierMotif
#print (modifierMotif("acg", 2))


def implantMotifVar(k, v, t, n, f):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    entrée f : frequence de variation du motif.
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    motif = generateRandomSequence(k, False)
    sequences = []
    
    for i in range(t):

        seq = generateRandomSequence(n-k)
        motif = generateRandomSequence(k, False) #pour mieux visualiser les motifs implantés on les inserts en minuscule
        #print("seq = " + seq)
        #print(motif)
        if random.uniform(0.0, 1.0) < f:
            motif = modifierMotif(motif, v)
        #print(motif)
        
        sequences.append(insertMotif(seq, motif, random.randint(0, len(seq)-1)))

    return sequences

adn = implantMotifVar(k, v, t, n, f)
print (adn)

adn  = [s.upper() for s in adn]
print (adn)

['GGCTCCATGCCcagtGCCATTATCGTTCATCTCATCTACGTGGAAGTAGCGCTATGTACGAATAGTCGAAGACGTACCGT', 'TCACAATCAGATCTGACCCGTAACACTGTATTAACAAGAGATACCTGGTAGTGTTTTCTgtgtcTGTCGGTGCTCTTCAT', 'TACTAGAATGGTTCCGATTTCACTCCATGCGGTTTCAACCGACACATGATCACGAGGTCCCAGCATCCTTctCcgGCCAG']
['GGCTCCATGCCCAGTGCCATTATCGTTCATCTCATCTACGTGGAAGTAGCGCTATGTACGAATAGTCGAAGACGTACCGT', 'TCACAATCAGATCTGACCCGTAACACTGTATTAACAAGAGATACCTGGTAGTGTTTTCTGTGTCTGTCGGTGCTCTTCAT', 'TACTAGAATGGTTCCGATTTCACTCCATGCGGTTTCAACCGACACATGATCACGAGGTCCCAGCATCCTTCTCCGGCCAG']


2\. Faire une fonction pour sélectionner des positions de départ aléatoirement `s = (s1, …,st)`.


In [4]:
#creating vector s
#Get t random positions from 0 to n - k
def generateRandomS(sequences, k):
	"""
	Génère un vecteur de position aléatoires
	entrée sequences: matrice de dimension txn contenant les sequences
	entrée k: taille du motif
	sortie s: vecteur de position aléatoires, une position par séquence
  REMARQUE les positions doivent être inférieur à n-k, ou n est la taille de sequence
	"""
	s = []

	for seq in sequences:
		if (len(seq) - k) > 0:
			pos= random.randint(0, len(seq)-k-1)
			s.append(pos)

	return s

s = generateRandomS(adn, k)
print("n = "+ str(n) + " k = " + str(k) +"\n")
print (s)

n = 80 k = 5

[57, 36, 14]


3\. Extraire les motifs en utilisant le vecteur `s`, et construire un profile (matrice de fréquence). 


In [5]:
#extract sequences from seqs
def extractSeqs(s, seqs, k):
    """
    Extraire les motifs des séquences à l'aide de positions s
    entrée s: vecteur contenant les positions de départs
    entrée seqs: matrice de dimension txn contenant les séquences
    entrée k: taille du motif
    sortie motifs: liste de motifs de taille k extrait des séquences
    """
    motifs = []
    for i in range(len(seqs)):
        sq = seqs[i]
        ind = s[i]
        motifs.append(sq[ind:ind + k])

    return motifs
        
motifs = extractSeqs(s, adn, k)
print("ADN : " + str(adn) + " s = " + str(s) + " k = " + str(k))
print (motifs )


#Construire un profile = matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension k x |nuc|
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    q = len(nuc)
    PWM = np.zeros((q, k))

    for mot in motifs:
        for i in range (k):
            indLigne = nuc.index(mot[i]) #indiceLigne
            PWM[indLigne,i]=PWM[indLigne,i]+1

    
  
    return PWM
MF = profile(motifs, k, nuc)
print("nuc = " + str(nuc))
print(MF)

ADN : ['GGCTCCATGCCCAGTGCCATTATCGTTCATCTCATCTACGTGGAAGTAGCGCTATGTACGAATAGTCGAAGACGTACCGT', 'TCACAATCAGATCTGACCCGTAACACTGTATTAACAAGAGATACCTGGTAGTGTTTTCTGTGTCTGTCGGTGCTCTTCAT', 'TACTAGAATGGTTCCGATTTCACTCCATGCGGTTTCAACCGACACATGATCACGAGGTCCCAGCATCCTTCTCCGGCCAG'] s = [57, 36, 14] k = 5
['ACGAA', 'AGAGA', 'CGATT']
nuc = ('A', 'C', 'G', 'T')
[[2. 0. 2. 1. 2.]
 [1. 1. 0. 0. 0.]
 [0. 2. 1. 1. 0.]
 [0. 0. 0. 1. 1.]]


4\. Transformer la matrice de fréquence en PWM. Il faut diviser chaque élément par la somme de ses colonnes.

In [6]:
def generatePWM(MF):
  """
  Transforme la matrice de fréquence en PWM
  entrée MF: matrice de fréquence
  sortie PWM: matrice de probabilité (poids positions)
  """
  PWM = MF

  nbligne = len(MF)
  nbcolonne = len(MF[0])
  sommes = [] #contient la somme de chaque colonne

  for i in range(nbcolonne): 
    s = 0 #tmp qui sauvegarde la somme de chaque colonne
    for j in range(nbligne):
      s = s + MF[j][i]
    sommes.append(s)

  for i in range(nbligne):
    for j in range(nbcolonne):
      PWM[i][j]=MF[i][j]/sommes[j]

  return PWM

PWM = generatePWM(MF)

print(PWM)

[[0.66666667 0.         0.66666667 0.33333333 0.66666667]
 [0.33333333 0.33333333 0.         0.         0.        ]
 [0.         0.66666667 0.33333333 0.33333333 0.        ]
 [0.         0.         0.         0.33333333 0.33333333]]


5\. Faire une fonction pour calculer la probabilité d'un motif de taille `k` selon une PWM.


In [7]:
def probability(PWM, motif):
    """
    Calcul la probalité d'un motif selon PWM
    entrée PWM: matrice de probabilité (poids positions)
    entrée motif: motif
    sortie prob: probalité Prob(motif|PWM)
    """
    prob = 1

    nuc = ('A', 'C', 'G', 'T')

    for colonne in range(len(motif)):
        indLigne = nuc.index(motif[colonne]) #indiceLigne
        prob = prob * PWM[indLigne,colonne]
    
    return prob

prob = probability(PWM, "AAAAA")
print (prob)

0.0


6\. Faire une fonction pour calculer le pMostProbkmer d'une séquence, voir un exemple dans les slides de cours.

In [8]:
def pMostProbkmer(PWM, k, sequence):
    """
    calcul la position du k-mer le plus probable dans la séquence
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: la position la plus probable 
    """
    maxProb = 0
    s = 0 # position du k-mer le plus probable dans la sequence

    for i in range(len(sequence) - k+1):
        tmp = probability(PWM,sequence[i:i+k]) #fenetre glissante sur le meme principe que la fonction searchMotif du tme 2 Hashtable
        if tmp > maxProb:
            maxProb = tmp
            s = i
    return s

s = pMostProbkmer(PWM, k, "GCCGGCCCAA")
print (s)

0


7\. Faire une fonction pour obtenir les nouvelles positions de départ `s = (s1, …,st)`, c’était à dire les positions qui contiens les k-mer le plus probables.

In [9]:
def getNewS(PWM, k, sequences):
    """
    Trouve les nouvelles positions des k-mer le plus probables
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: vecteur avec les positions les plus problables
    """

    s = []
    for seq in sequences:
          s.append(pMostProbkmer(PWM, k, seq))

    return s

ns = getNewS(PWM, k, adn)
print (ns)

[57, 36, 4]


8\. La condition d’arrêt de l'algorithme est le non changement de la matrice de fréquence d'une itération à l'autre, 
faire une fonction pour comparer deux matrices et détecter le changement. 
Faire aussi une fonction pour obtenir le score d'une matrice de fréquence : la somme de max de chaque colonne

In [10]:
def changeProfile(P1, P2):
    """
    Compare deux matrice
    entrée P1: matrice de fréquence
    entrée P2: matrice de fréquence
    sortie: True si les matrices sont différents, False au contraire
    """
    nbLigne=min(len(P1),len(P2))
    nbColonne=min(len(P1[0]),len(P2[0]))

    #if(len(P1)!=len(P2) or len(P1[0]) != len(P2[0])): #si elle ne font pas la même taille, elles sont forcement differente (jsp si faut tester ça), inutile si on suppose quelle font tjr la meme
        #return True 
    
    nbligne = min(len(P1),len(P2))
    nbcol = min(len(P1[0]),len(P2[0]))

    for l in range(nbligne):
        for c in range(nbcol):
            if P1[l,c] != P2[l,c]:
                return True
   
    return False

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    sc = 0
    nbLigne = len(MF)
    nbColonne = len(MF[0])

    for c in range(nbColonne):
        max = 0
        for l in range(nbLigne):
            if MF[l, c] > max:
                max = MF[l, c]
        #print(max)
        sc = sc + max
    
    return sc

ns = getNewS(PWM, k, adn)
print (s, ns)

sc = getScore(MF, k)
print (sc)

MFtest1=np.array([[0, 0, 0, 0, 0],[0, 0, 0, 0, 0],[0, 0, 0, 0, 0],[0, 0, 0, 0, 0]])
MFtest2=np.array([[0, 0, 0, 0, 0],[0, 0, 0, 0, 0],[0, 0, 0, 0, 0],[0, 0, 0, 0, 0]])

print(changeProfile(MFtest1, MFtest2))

#print(MF)


0 [57, 36, 4]
3.0
False


9\. Implementer l'algorithme ``GreedyProfileMotifSearch`` en utilisant les fonctions precedentes. 
BONUS: Pour ameliorer la performance vous pouvez abandonner les positions qui génèrent des motifs peu complexes.

In [11]:
def GreedyProfileMotifSearch(sequences, k):
    """
    Implémente l'algorithme GreedyProfileMotifSearch 
	  entrée sequences: matrice de dimension txn contenant les séquences 
	  entrée k : taille du motif 
    sortie s : vecteur de positions de départ ayant le meilleur motif 
    sortie bestScore : le score associé à s
    """
    s = generateRandomS(sequences, k)
    bestScore = 0
    
    nuc = ('A', 'C', 'G', 'T')
    
    motifs=extractSeqs(s, sequences, k) #Liste des motifs plus probable
    MF=profile(motifs, k, nuc) #Détermine le Profile/MF
    PWM=generatePWM(MF) #Détermine la matrice de proba associée
    score =getScore(MF,k) #Score associé à la MF courante déterminer par les motifs les plus probables(P-most probable k-mers) jusqu'à présent obtenue
    
    #Variables temporaires pour les itérations
    tmpS=getNewS(PWM, k, sequences)
    tmpMotifs=extractSeqs(tmpS, sequences, k)
    tmpMF=profile(tmpMotifs, k, nuc)
    
    while score>bestScore:
        bestScore= score
        
        if (changeProfile(MF, tmpMF)): #Profil sont différents
            #Cas bestscore < score : on continue l'algo
            if getScore(tmpMF,k) > bestScore:
                
                #Màj des paramètres liés au motif les plus probables de nos séquances
                bestScore=getScore(tmpMF,k)
                motifs=tmpMotifs
                MF=tmpMF
                PWM=generatePWM(MF) #Génère une new matrice de proba pour déterminer une nouvelle matrice de position
    
                #Anticipation de la prochaine itération en mettant à jour les variables temporaires
                tmpS=getNewS(PWM, k, sequences)
                tmpMotifs=extractSeqs(tmpS, sequences, k)
                tmpMF=profile(tmpMotifs, k, nuc)
                score=getScore(tmpMF,k)
 
            #cas score < bestscore
            else :
                break
        
        else: #Profil ont pas change on stop l'algo entre 2 itérations
            break
              
    
    return s, bestScore

print (adn)
s, bestScore = GreedyProfileMotifSearch(adn, k)   
motifs = extractSeqs(s, adn, k)
print (motifs)
        

['GGCTCCATGCCCAGTGCCATTATCGTTCATCTCATCTACGTGGAAGTAGCGCTATGTACGAATAGTCGAAGACGTACCGT', 'TCACAATCAGATCTGACCCGTAACACTGTATTAACAAGAGATACCTGGTAGTGTTTTCTGTGTCTGTCGGTGCTCTTCAT', 'TACTAGAATGGTTCCGATTTCACTCCATGCGGTTTCAACCGACACATGATCACGAGGTCCCAGCATCCTTCTCCGGCCAG']
['TTATC', 'TCACA', 'CTAGA']


10\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

Reponse:

11\. Vous avez certainement observer que l’algorithme ne produire pas toujours la même sortie et que les résultats dépendent de la sélection aléatoire des positions de départ `s`. Pour augmenter nous chances de retrouvez les bons motifs, nous allons implémenter une version itérative ``GreedyProfileMotifSearchIte`` 
qui a chaque fois sauvegarde `s` et le score du profile associé à `s`, après `I` itération, l’algorithme renvoie le vecteur `s` ayant le plus grand score.

In [12]:
def GreedyProfileMotifSearchIte(sequences, k, It):
    """
    Implémente l'algorithme GreedyProfileMotifSearch iteratif
	  entrée séquences: matrice de dimension txn contenant les séquences
	  entrée k: taille du motif 
    entrée It: nombre d'iterations
    sortie positions: dictionnaire clé=score, valeur= vecteur s 
    sortie consensus: dictionnaire clé=score, valeur= sequence consensus du motif
    """
    
    pos = {}
    consensus = {}
    for i in range(It):
        (s, bestScore) = GreedyProfileMotifSearch(sequences, k)
        pos[bestScore]=s
        consensus[bestScore]=extractSeqs(s, sequences, k)
   
    return pos, consensus


s, consensus = GreedyProfileMotifSearchIte(adn, k, 50)

print (s)
print (consensus)

{13.0: [66, 44, 71], 10.0: [62, 41, 51], 12.0: [7, 62, 1], 11.0: [57, 5, 63], 14.0: [40, 69, 23]}
{13.0: ['CGAAG', 'CTGGT', 'TCCGG'], 10.0: ['TAGTC', 'TACCT', 'ACGAG'], 12.0: ['TGCCC', 'TCTGT', 'ACTAG'], 11.0: ['ACGAA', 'ATCAG', 'CATCC'], 14.0: ['TGGAA', 'GTGCT', 'TCCAT']}


In [13]:
#Affiche les sequence consensus de top motifs
def printTopFMotifsScore(consensus, top):

    motifsSort = sorted(consensus.keys())
    motifsSort.reverse()
    for i in range(top):
      print (consensus[motifsSort[i]])

printTopFMotifsScore(consensus, 2)




['TGGAA', 'GTGCT', 'TCCAT']
['CGAAG', 'CTGGT', 'TCCGG']


12\. Tester algorithme  ``GreedyProfileMotifSearchIte`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats.

In [14]:

def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

In [25]:
k=8

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_9.fasta"


sequencesChip = readFasta(genome, n)
t = len(sequencesChip)
print (sequencesChip[8], t, k)
revSequences = [reverseComplement(m) for m in sequencesChip]

sequencesChip = sequencesChip + revSequences
s, consensus = GreedyProfileMotifSearchIte(sequencesChip, k, 20)
printTopFMotifsScore(consensus, 10)


CTTCCCCAAGCCCAAGCCCAAGCCCAATGGCGTATTATCCTGAACTTCCCAGCCGAGCCCCTAGGCCCCCCCCCCACCCG 174 8
['GGGTAGCC', 'AAACCAGT', 'AGAAAAAA', 'TACGTTCT', 'AAAAATTA', 'TTGAAATG', 'AAATTAAA', 'CTAATAAA', 'GCCCAAGC', 'TGGCCCAG', 'CGGGTACA', 'GGACGAGT', 'GAGGTCGG', 'GTGAGCAT', 'GGGATGGA', 'AAGAGAAA', 'CCCAATTG', 'TCAGAGAG', 'TCTTCCAT', 'TCTGCAAG', 'TGGAAGGA', 'GAGCTACC', 'TTTGCACC', 'CGCAGGAT', 'AATTATGT', 'GCGTCTAA', 'AGACACCT', 'CACCACTA', 'TAGAAGGA', 'AATGCGTG', 'TAGCGTAG', 'AAGTAGAG', 'GAAACAAA', 'ACATCCCG', 'CAGTGGTG', 'ATTATTCC', 'GTTTCTCC', 'AGAATGCT', 'CAGATTCT', 'AGAAAAAG', 'GAAGAGGG', 'ATAACCTT', 'CAGAATGC', 'GCAAGTTG', 'TGGGGGTA', 'TCCATGAT', 'GTGATGGA', 'GAGCACAC', 'ACGCTACA', 'TCAACGGT', 'CTACTCCT', 'AAAGATTG', 'GATGTTAT', 'CATTACCT', 'TCTTGGCA', 'CTAAAAAT', 'GTTGCATT', 'ATGGTGCT', 'GGTAGGCC', 'ACAAAGTC', 'AGAAATGG', 'TCAGTGTC', 'GGAAGCTA', 'GGCATTGT', 'TAGAGACA', 'AAATTGCT', 'TATGAGTT', 'GTTATTGA', 'TACATAGT', 'TAGGAACA', 'CTGTTAGC', 'GACAAAAT', 'AACTACGA', 'TCTATCCC', 'TGATATAA', 'CACAGAGA', 